# Python Packaging

```
@crlane & @sduncan
Tea Time 2017-03-16
```

## Some Terminology

- *distribution* - a collection of one or more _packages_ intended for release, either in archive (bdist, wheel), or source (sdist) form.

- *package* - an installable collection of one or more python _modules_, data, and associated metadata

- *module* - an importable unit of executable python code, either pure python or extension (C/C++)

## Some History

- python is kind of old; there is more than one way to do it. None of them is obvious.

- packaging is somewhat opaque, but there are *well defined standards*. Follow them.

- `setuptools > distutils`

- `pip install > python setup.py install`

- `wheel > egg`

## Some Examples

### How to define a simple python distribution

1. Write some code in a `.py` file 

In [2]:
import requests
import sys


def get_url(url):
    session = requests.Session()
    return session.get(url)


def command_line():
    if not len(sys.argv) > 1:
        print("Please supply a url to fetch!")
        sys.exit(1)
    return url_getter(sys.argv[1])

1. Create an executable specification for building the package (setup.py)
   - What packages exist in the distribution?
   - What are the package's dependencies (for install, setup, testing)?
   - What non executable data should be included (text files, static assets, templates)?
   - Public info (maintainer contact, description, licences...)

In [ ]:
from setuptools import (
    setup,
    find_packages  # what it says on the tin
)

setup(
    name='urlfetch',
    version='0.1.0',
    description='Fetches URLs! Web browsers hate it!',
    install_requires=['requests'],  # could also read a requirements.txt file to populate list
    packages=find_packages(),  # can also explicitly list things
    license='wtfpl',
    entry_points={  # you can create really cool things here
        'console_scripts': [
            'fetch_url=urlgetter:command_line'
        ]
    },
)

### How to package the distribution so someone else can use it

- source distribution (tar.gz) of your code.

```
python setup.py sdist
```

### sdist

```
(packaging-talk)➜  urlfetch-0.1.0 git:(master) ✗ tree
.
├── PKG-INFO
├── setup.cfg
├── setup.py
└── urlfetch.egg-info
    ├── PKG-INFO
    ├── SOURCES.txt
    ├── dependency_links.txt
    ├── entry_points.txt
    ├── requires.txt
    └── top_level.txt
```

- binary distribution (platform specific)

```
python setup.py bdist
```

### bdist

```
(packaging-talk)➜  dist git:(master) ✗ tar -xvzf urlfetch-0.1.0.macosx-10.11-x86_64.tar.gz
x ./
x ./Users/
x ./Users/clane/
x ./Users/clane/.virtualenvs/
x ./Users/clane/.virtualenvs/packaging-talk/
x ./Users/clane/.virtualenvs/packaging-talk/bin/
x ./Users/clane/.virtualenvs/packaging-talk/bin/fetch_url
x ./Users/clane/.virtualenvs/packaging-talk/lib/
x ./Users/clane/.virtualenvs/packaging-talk/lib/python3.6/
x ./Users/clane/.virtualenvs/packaging-talk/lib/python3.6/site-packages/
x ./Users/clane/.virtualenvs/packaging-talk/lib/python3.6/site-packages/urlfetch-0.1.0-py3.6.egg-info/
x ./Users/clane/.virtualenvs/packaging-talk/lib/python3.6/site-packages/urlfetch-0.1.0-py3.6.egg-info/dependency_links.txt
x ./Users/clane/.virtualenvs/packaging-talk/lib/python3.6/site-packages/urlfetch-0.1.0-py3.6.egg-info/entry_points.txt
x ./Users/clane/.virtualenvs/packaging-talk/lib/python3.6/site-packages/urlfetch-0.1.0-py3.6.egg-info/PKG-INFO
x ./Users/clane/.virtualenvs/packaging-talk/lib/python3.6/site-packages/urlfetch-0.1.0-py3.6.egg-info/requires.txt
x ./Users/clane/.virtualenvs/packaging-talk/lib/python3.6/site-packages/urlfetch-0.1.0-py3.6.egg-info/SOURCES.txt
x ./Users/clane/.virtualenvs/packaging-talk/lib/python3.6/site-packages/urlfetch-0.1.0-py3.6.egg-info/top_level.txt
```

- binary wheel distribution

```
python setup.py bdist_wheel
```

### wheel

```
(packaging-talk)➜  urlfetch-0.1.0.dist-info git:(master) ✗ tree
.
├── DESCRIPTION.rst
├── METADATA
├── RECORD
├── WHEEL
├── entry_points.txt
├── metadata.json
└── top_level.txt

0 directories, 7 files
```

- rpm distribution (For RHEL/Centos)

```
python setup.py bdist_rpm
```

### bdist_rpm

```
(packaging-talk)➜  dist git:(master) ✗ tar -tvzf urlfetch-0.1.0-1.src.rpm
-rw-r--r--  1 0      0         934 Mar 16 14:56 urlfetch-0.1.0.tar.gz
-rw-r--r--  1 0      0         775 Mar 16 14:56 urlfetch.spec
```

### Some Advanced Features

- `extra_requires`

```python
    extras_require={
        'testing': tests_require,
        'development': dev_requires,
    },

```

- extension building for C/C++ (including Cython)

```python
setup(
    name='fds_ss',
    packages=find_packages(exclude=[
        '*.test',
        '*.test.*',
        'test.*',
        'test',
        '.ignored'
    ]),
    version='VERSION',
    cmdclass={
        'build_ext': build_ext
    },
    ext_modules=[
        Extension(
            'fds.netaudio',
            sources=['fds/_netaudio/netaudio.pyx'],
            include_dirs=['fds/_netaudio'],
        )
    ],
    entry_points={
        # IMPORTANT: In order for these console scripts to be on the system PATH for
        # packaging reasons, you MUST add the symlink to debian/python-fds-ss.links
        'console_scripts': [
            'correlate_call_org = fds.lob.cli.correlate_call_org:main',
            'fds-admin          = fds.cli.fds_admin:FDSAdminCommand.main',
            ... snip ...
        ]
    })
```

- `pkg_resources` for plugins

```python
import pkg_resources  # Part of setuptools

# Give me the names of everything in the foo.bar namespace
names = set(ep.name for ep in pkg_resources.iter_entry_points('foo.bar'))

# Dynamically load modules registered in the foo.bar.namespace
for ep in pkg_resources.iter_entry_points(group='foo.bar'):
    # ep.load() handles normally what __import__ or importlib would without a lot of fuss
    dingus = ep.load()

    # dingus is now a module/class/function whatever
```

- stdeb

## Some Links

- https://www.pypa.io/en/latest/
- https://packaging.python.org/
- http://wheel.readthedocs.io/en/latest/
- https://github.com/shaunduncan/helga/blob/master/setup.py